<a href="https://colab.research.google.com/github/vfrantc/dehaze_detect/blob/main/oldmodel_combo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks.git

  Cloning https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks.git to /tmp/pip-req-build-29_ogv8s
  Running command git clone -q https://github.com/Orkis-Research/Pytorch-Quaternion-Neural-Networks.git /tmp/pip-req-build-29_ogv8s
  Created wheel for Pytorch-QNN: filename=Pytorch_QNN-1-py3-none-any.whl size=21517 sha256=bb4ffe97208392fa2b20f305d44332277e4fd95e793ed9a1cd34d1641854a2df
  Stored in directory: /tmp/pip-ephem-wheel-cache-tn1slipk/wheels/28/96/bc/2d440ce957d6b5ce8d0345b758b7828d07fae2c5a9f3fae8c7
Successfully built Pytorch-QNN


In [2]:
!nvidia-smi

Mon Nov 22 10:00:55 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
import datetime
import argparse
import numpy as np

import cv2
import torch
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from scipy import signal
from torchvision.utils import make_grid
import numpy.random as random

from PIL import Image

from core_qnn.quaternion_layers import QuaternionTransposeConv
from core_qnn.quaternion_layers import QuaternionConv
from core_qnn.quaternion_ops import check_input
from core_qnn.quaternion_ops import get_r, get_i, get_j, get_k

In [4]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [5]:
!cp /content/drive/MyDrive/dataset2.zip .
!unzip -q dataset2.zip
!mv I-HAZE/hazy .
!rm -rf I-HAZE
!mv hazy I-HAZE 
!mv O-HAZE/hazy . 
!rm -rf O-HAZE 
!mv hazy O-HAZE

In [6]:
!mv SOTS/OUTDOOR/hazy ./SOTS-OUTDOOR
!mv SOTS/INDOOR/hazy ./SOTS-INDOOR

In [7]:
!cp /content/drive/MyDrive/nohomo2.zip .
!unzip nohomo2.zip

Streaming output truncated to the last 5000 lines.
  inflating: nohomo/outdoor/train/gt/o2781.jpg  
  inflating: nohomo/outdoor/train/gt/o2048.jpg  
  inflating: nohomo/outdoor/train/gt/o3562.jpg  
  inflating: nohomo/outdoor/train/gt/o1905.jpg  
  inflating: nohomo/outdoor/train/gt/o2558.jpg  
  inflating: nohomo/outdoor/train/gt/o2313.jpg  
  inflating: nohomo/outdoor/train/gt/o3942.jpg  
  inflating: nohomo/outdoor/train/gt/o5821.jpg  
  inflating: nohomo/outdoor/train/gt/o5416.jpg  
  inflating: nohomo/outdoor/train/gt/o4860.jpg  
  inflating: nohomo/outdoor/train/gt/o4557.jpg  
  inflating: nohomo/outdoor/train/gt/o2921.jpg  
  inflating: nohomo/outdoor/train/gt/o4236.jpg  
  inflating: nohomo/outdoor/train/gt/o2463.jpg  
  inflating: nohomo/outdoor/train/gt/o2379.jpg  
  inflating: nohomo/outdoor/train/gt/o0782.jpg  
  inflating: nohomo/outdoor/train/gt/o4874.jpg  
  inflating: nohomo/outdoor/train/gt/o1912.jpg  
  inflating: nohomo/outdoor/train/gt/o4909.jpg  
  inflating: nohom

In [8]:
!mv ITS/clear ITS/gt

In [9]:
from glob import glob 
from tqdm.notebook import tqdm
import shutil

for fname in tqdm(glob('ITS/gt/*')):
  out = os.path.join(os.path.dirname(fname), 't'+os.path.basename(fname))
  shutil.move(fname, out)

for fname in tqdm(glob('ITS/hazy/*')):
  out = os.path.join(os.path.dirname(fname), 't'+os.path.basename(fname))
  shutil.move(fname, out)

  0%|          | 0/1399 [00:00<?, ?it/s]

  0%|          | 0/13990 [00:00<?, ?it/s]

In [10]:
!cp -r ITS/gt OTS/ 
!cp -r ITS/hazy OTS/

In [11]:
!rm -rf SOTS-INDOOR SOTS-OUTDOOR SOTS 

In [12]:
!cp /content/drive/MyDrive/SOTS.zip .

In [13]:
!unzip SOTS.zip

Archive:  SOTS.zip
   creating: SOTS/
   creating: SOTS/indoor/
   creating: SOTS/indoor/gt/
 extracting: SOTS/indoor/gt/1400.png  
 extracting: SOTS/indoor/gt/1401.png  
 extracting: SOTS/indoor/gt/1402.png  
 extracting: SOTS/indoor/gt/1403.png  
 extracting: SOTS/indoor/gt/1404.png  
 extracting: SOTS/indoor/gt/1405.png  
 extracting: SOTS/indoor/gt/1406.png  
 extracting: SOTS/indoor/gt/1407.png  
 extracting: SOTS/indoor/gt/1408.png  
 extracting: SOTS/indoor/gt/1409.png  
 extracting: SOTS/indoor/gt/1410.png  
 extracting: SOTS/indoor/gt/1411.png  
 extracting: SOTS/indoor/gt/1412.png  
 extracting: SOTS/indoor/gt/1413.png  
 extracting: SOTS/indoor/gt/1414.png  
 extracting: SOTS/indoor/gt/1415.png  
 extracting: SOTS/indoor/gt/1416.png  
 extracting: SOTS/indoor/gt/1417.png  
 extracting: SOTS/indoor/gt/1418.png  
 extracting: SOTS/indoor/gt/1419.png  
 extracting: SOTS/indoor/gt/1420.png  
 extracting: SOTS/indoor/gt/1421.png  
 extracting: SOTS/indoor/gt/1422.png  
 extractin

# Utils

In [14]:
IMG_EXTENSIONS = [
    '.jpg', '.JPG', '.jpeg', '.JPEG',
    '.png', '.PNG', '.ppm', '.PPM', '.bmp', '.BMP',
]

In [15]:
def is_image_file(filename):
    return any(filename.endswith(extension) for extension in IMG_EXTENSIONS)

In [16]:
def make_dataset(dir):
    images = []
    assert os.path.isdir(dir), '%s is not a valid directory' % dir

    for root, _, fnames in sorted(os.walk(dir)):
        for fname in fnames:
            if is_image_file(fname):
                path = os.path.join(root, fname)
                images.append(path)

    return images

In [17]:
def edge_compute(x):
    x_diffx = torch.abs(x[:,:,1:] - x[:,:,:-1])
    x_diffy = torch.abs(x[:,1:,:] - x[:,:-1,:])

    y = x.new(x.size())
    y.fill_(0)
    y[:,:,1:] += x_diffx
    y[:,:,:-1] += x_diffx
    y[:,1:,:] += x_diffy
    y[:,:-1,:] += x_diffy
    y = torch.sum(y,0,keepdim=True)/3
    y /= 4
    return y

In [18]:
def batch_edge_compute(x):
    x_diffx = torch.abs(x[:,:,:,1:] - x[:,:,:,:-1])
    x_diffy = torch.abs(x[:,:,1:,:] - x[:,:,:-1,:])

    y = x.new(x.size())
    y.fill_(0)
    y[:,:,:,1:] += x_diffx
    y[:,:,:,:-1] += x_diffx
    y[:,:,1:,:] += x_diffy
    y[:,:,:-1,:] += x_diffy
    y = torch.sum(y,1,keepdim=True)/3
    y /= 4
    return y

In [19]:
# Converts a Tensor into an image array (numpy)
# |imtype|: the desired type of the converted numpy array
def tensor2im(input_image, imtype=np.uint8):
    if isinstance(input_image, torch.Tensor):
        image_tensor = input_image.data
    else:
        return input_image
    image_numpy = image_tensor[0].cpu().float().numpy()
    if image_numpy.shape[0] == 1:
        image_numpy = np.tile(image_numpy, (3, 1, 1))
    image_numpy = (np.transpose(image_numpy, (1, 2, 0)) + 1) / 2.0 * 255.0
    image_numpy = image_numpy.clip(0, 255)
    return image_numpy.astype(imtype)

In [20]:
def tensor2imgrid(input_image):
    im_grid = make_grid(input_image[:4, ...], nrow=2, normalize=True, range=(-128, 128))
    return im_grid
    # ndarr = im_grid.mul(255).clamp(0, 255).byte().permute(1, 2, 0).cpu().numpy()
    # im = Image.fromarray(ndarr)
    # return im

In [21]:
def diagnose_network(net, name='network'):
    mean = 0.0
    count = 0
    for param in net.parameters():
        if param.grad is not None:
            mean += torch.mean(torch.abs(param.grad.data))
            count += 1
    if count > 0:
        mean = mean / count
    print(name)
    print(mean)

In [22]:
def save_image(image_numpy, image_path):
    image_pil = Image.fromarray(image_numpy)
    image_pil.save(image_path)

In [23]:
def print_numpy(x, val=True, shp=False):
    x = x.astype(np.float64)
    if shp:
        print('shape,', x.shape)
    if val:
        x = x.flatten()
        print('mean = %3.3f, min = %3.3f, max = %3.3f, median = %3.3f, std=%3.3f' % (
            np.mean(x), np.min(x), np.max(x), np.median(x), np.std(x)))

In [24]:
def mkdirs(paths):
    if isinstance(paths, list) and not isinstance(paths, str):
        for path in paths:
            mkdir(path)
    else:
        mkdir(paths)

In [25]:
def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [26]:
def fspecial_gauss(size, sigma):
    """Function to mimic the 'fspecial' gaussian MATLAB function
    """
    x, y = np.mgrid[-size // 2 + 1:size // 2 + 1, -size // 2 + 1:size // 2 + 1]
    g = np.exp(-((x ** 2 + y ** 2) / (2.0 * sigma ** 2)))
    return g / g.sum()

In [27]:
def filter2(x, kernel, mode='same'):
    return signal.convolve2d(x, np.rot90(kernel, 2), mode=mode)

In [28]:
def ssim(img1, img2, cs_map=False):
    """Return the Structural Similarity Map corresponding to input images img1
    and img2 (images are assumed to be uint8)

    This function attempts to mimic precisely the functionality of ssim.m a
    MATLAB provided by the author's of SSIM
    https://ece.uwaterloo.ca/~z70wang/research/ssim/ssim_index.m
    """
    img1 = img1.astype(np.float64)
    img2 = img2.astype(np.float64)
    size = 11
    sigma = 1.5
    window = fspecial_gauss(size, sigma)
    K1 = 0.01
    K2 = 0.03
    L = 255  # bitdepth of image
    C1 = (K1 * L) ** 2
    C2 = (K2 * L) ** 2
    mu1 = filter2(img1, window, mode='valid')
    mu2 = filter2(img2, window, mode='valid')
    mu1_sq = mu1 * mu1
    mu2_sq = mu2 * mu2
    mu1_mu2 = mu1 * mu2
    sigma1_sq = filter2(img1 * img1, window, mode='valid') - mu1_sq
    sigma2_sq = filter2(img2 * img2, window, mode='valid') - mu2_sq
    sigma12 = filter2(img1 * img2, window, mode='valid') - mu1_mu2
    if cs_map:
        return np.mean(np.mean((((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                             (sigma1_sq + sigma2_sq + C2)),
                (2.0 * sigma12 + C2) / (sigma1_sq + sigma2_sq + C2))))
    else:
        return np.mean(np.mean(((2 * mu1_mu2 + C1) * (2 * sigma12 + C2)) / ((mu1_sq + mu2_sq + C1) *
                                                            (sigma1_sq + sigma2_sq + C2))))

In [29]:
class MovingAvg(object):
    def __init__(self, pool_size=100):
        from queue import Queue
        self.pool = Queue(maxsize=pool_size)
        self.sum = 0
        self.curr_pool_size = 0
        self.pool_size = pool_size

    def set_curr_val(self, val):
        if not self.pool.full():
            self.curr_pool_size += 1
            self.pool.put_nowait(val)
        else:
            last_first_val = self.pool.get_nowait()
            self.pool.put_nowait(val)
            self.sum -= last_first_val

        self.sum += val
        return self.sum / self.curr_pool_size

    def reset(self):
        from queue import Queue
        self.pool = Queue(maxsize=self.pool_size)
        self.sum = 0
        self.curr_pool_size = 0

# Folder loader

In [30]:
class FolderLoader(object):
    def __init__(self, fold_path):
        super(FolderLoader, self).__init__()
        self.fold_path = fold_path
        self.img_paths = make_dataset(self.fold_path)
        self.img_names = [os.path.basename(x) for x in self.img_paths]

    def __getitem__(self, index):
        img = Image.open(self.img_paths[index])#.convert('RGB')
        return self.img_names[index], img

    def __len__(self):
        return len(self.img_names)

In [31]:
def pil_loader(img_path):
    return Image.open(img_path).convert("RGB")

In [32]:
class ImagePairPrefixFolder(Dataset):
    def __init__(self, input_folder, gt_folder, max_img_size=0, size_unit=1, force_rgb=False):
        super(ImagePairPrefixFolder, self).__init__()

        self.gt_loader = FolderLoader(gt_folder)
        # build the map from image name to index
        self.gt_map = dict()
        for idx, img_name in enumerate(self.gt_loader.img_names):
            self.gt_map[os.path.splitext(img_name)[0].split('_')[0]] = idx

        self.input_loader = FolderLoader(input_folder)
        assert all([os.path.splitext(x)[0].split('_')[0] in self.gt_map for x in self.input_loader.img_names]), \
                'cannot find corresponding gt names'


        self.input_folder = input_folder
        self.gt_folder = gt_folder
        self.max_img_size = max_img_size
        self.size_unit = size_unit
        self.force_rgb = force_rgb

    def __getitem__(self, index):
        input_name, input_img = self.input_loader[index]
        input_basename = os.path.splitext(input_name)[0].split('_')[0]
        gt_idx = self.gt_map[input_basename]

        gt_name, gt_img = self.gt_loader[gt_idx]
        if self.force_rgb:
            input_img = input_img.convert('RGB')
            gt_img = gt_img.convert('RGB')
        im_w, im_h = input_img.size
        gt_w, gt_h = gt_img.size

        if (im_w != gt_w) or (im_h != gt_h):
          print(input_name)
          print(gt_name)

        assert im_w==gt_w and im_h==gt_h, 'input image and gt image size not match'

        im_w, im_h = input_img.size
        if 0 < self.max_img_size < max(im_w, im_h):
            if im_w < im_h:
                out_h = int(self.max_img_size) // self.size_unit * self.size_unit
                out_w = int(im_w / im_h * out_h) // self.size_unit * self.size_unit
            else:
                out_w = int(self.max_img_size) // self.size_unit * self.size_unit
                out_h = int(im_h / im_w * out_w) // self.size_unit * self.size_unit
        else:
            out_w = im_w // self.size_unit * self.size_unit
            out_h = im_h // self.size_unit * self.size_unit

        if im_w != out_w or im_h != out_h:
            input_img = input_img.resize((out_w, out_h), Image.BILINEAR)
            gt_img = gt_img.resize((out_w, out_h), Image.BILINEAR)

        im_w, im_h = input_img.size

        
        #input_img = np.array(input_img).astype('float')
        #input_img = np.array(input_img)

        input_img = np.array(input_img)
        gray = cv2.cvtColor(input_img, cv2.COLOR_RGB2GRAY)
        #gray = cv2.imread(os.path.join('ds/train_general/trans/', input_name.replace('.jpg', '.png')), 0)
        #gray = cv2.resize(gray, (out_w, out_h))
        input_img = np.dstack([gray[:, :, np.newaxis], input_img])
        input_img = input_img.astype('float')


        gt_img = np.array(gt_img)
        gray = cv2.cvtColor(gt_img, cv2.COLOR_RGB2GRAY)
        gt_img = np.dstack([gray[:, :, np.newaxis], gt_img])
        gt_img = gt_img.astype('float')
        if len(input_img.shape) == 2:
            input_img = input_img[:, :, np.newaxis]
        if len(gt_img.shape) == 2:
            gt_img = gt_img[:, :, np.newaxis]

        input_img = input_img
        gt_img = gt_img
        return {'input_img': input_img, 'gt_img': gt_img,  'input_h': im_h, "input_w": im_w}

    def get_input_info(self, index):
        image_name = os.path.splitext(self.input_loader.img_names[index])[0]
        return self.input_loader, image_name

    def __len__(self):
        return len(self.input_loader)

In [33]:
def var_custom_collate(batch):
    min_h, min_w = 10000, 10000
    for item in batch:
        min_h = min(min_h, item['input_h'])
        min_w = min(min_w, item['input_w'])
    inc = 1 if len(batch[0]['input_img'].shape)==2 else batch[0]['input_img'].shape[2]
    batch_input_images = torch.Tensor(len(batch), 4, min_h, min_w)
    batch_gt_images = torch.Tensor(len(batch), 4, min_h, min_w)

    for idx, item in enumerate(batch):
        off_y = 0 if item['input_h']==min_h else random.randint(0, item['input_h'] - min_h)
        off_x = 0 if item['input_w']==min_w else random.randint(0, item['input_w'] - min_w)
        crop_input_img = item['input_img'][off_y:off_y + min_h, off_x:off_x + min_w, :]
        crop_gt_img = item['gt_img'][off_y:off_y + min_h, off_x:off_x + min_w, :]
        batch_input_images[idx] = torch.from_numpy(crop_input_img.transpose((2, 0, 1))) - 128
        batch_gt_images[idx] = torch.from_numpy(crop_gt_img.transpose((2, 0, 1)))


    batch_input_edges = batch_edge_compute(batch_input_images) - 128
    return batch_input_images, batch_input_edges,  batch_gt_images

# Network

In [34]:
class ShareSepConv(nn.Module):
    def __init__(self, kernel_size):
        super(ShareSepConv, self).__init__()
        assert kernel_size % 2 == 1, 'kernel size should be odd'
        self.padding = (kernel_size - 1)//2
        weight_tensor = torch.zeros(1, 1, kernel_size, kernel_size)
        weight_tensor[0, 0, (kernel_size-1)//2, (kernel_size-1)//2] = 1
        self.weight = nn.Parameter(weight_tensor)
        self.kernel_size = kernel_size

    def forward(self, x):
        inc = x.size(1)
        expand_weight = self.weight.expand(inc, 1, self.kernel_size, self.kernel_size).contiguous()
        return F.conv2d(x, expand_weight,
                        None, 1, self.padding, 1, inc)

In [35]:
class QuaternionInstanceNorm2d(nn.Module):
    def __init__(self, num_features, gamma_init=1., beta_param=True, training=True):
        super(QuaternionInstanceNorm2d, self).__init__()
        self.num_features = num_features // 4
        self.gamma_init = gamma_init
        self.beta_param = beta_param
        self.gamma = nn.Parameter(torch.full([1, self.num_features, 1, 1], self.gamma_init))
        self.beta = nn.Parameter(torch.zeros(1, self.num_features * 4, 1, 1), requires_grad=self.beta_param)
        self.training = training
        self.eps = torch.tensor(1e-5)

    def reset_parameters(self):
        self.gamma = nn.Parameter(torch.full([1, self.num_features, 1, 1], self.gamma_init))
        self.beta = nn.Parameter(torch.zeros(1, self.num_features * 4, 1, 1), requires_grad=self.beta_param)

    def forward(self, input):
        quat_components = torch.chunk(input, 4, dim=1)
        r, i, j, k = quat_components[0], quat_components[1], quat_components[2], quat_components[3]
        delta_r, delta_i, delta_j, delta_k = r - torch.mean(r, axis=[1, 2, 3], keepdim=True), i - torch.mean(i, axis=[1, 2, 3], keepdim=True), j - torch.mean(j, axis=[1, 2, 3], keepdim=True), k - torch.mean(k, axis=[1, 2, 3], keepdim=True)
        quat_variance = torch.mean((delta_r**2 + delta_i**2 + delta_j**2 + delta_k**2))
        denominator = torch.sqrt(quat_variance + self.eps)

        # Normalize
        r_normalized = delta_r / denominator
        i_normalized = delta_i / denominator
        j_normalized = delta_j / denominator
        k_normalized = delta_k / denominator

        beta_components = torch.chunk(self.beta, 4, dim=1)

        # Multiply gamma (stretch scale) and add beta (shift scale)
        new_r = (self.gamma * r_normalized) + beta_components[0]
        new_i = (self.gamma * i_normalized) + beta_components[1]
        new_j = (self.gamma * j_normalized) + beta_components[2]
        new_k = (self.gamma * k_normalized) + beta_components[3]

        new_input = torch.cat((new_r, new_i, new_j, new_k), dim=1)

        return new_input

    def __repr__(self):
        return self.__class__.__name__ + '(' \
               + 'num_features=' + str(self.num_features) \
               + ', gamma=' + str(self.gamma) \
               + ', beta=' + str(self.beta) \
               + ', eps=' + str(self.eps) + ')'

In [36]:
class SmoothDilatedResidualBlock(nn.Module):
    def __init__(self, channel_num, dilation=1, group=1):
        super(SmoothDilatedResidualBlock, self).__init__()
        self.pre_conv1 = ShareSepConv(dilation*2-1)
        self.conv1 = QuaternionConv(channel_num, channel_num, 3, 1, padding=dilation, dilatation=dilation, groups=group, bias=False)
        self.norm1 = QuaternionInstanceNorm2d(channel_num)
        self.pre_conv2 = ShareSepConv(dilation*2-1)
        self.conv2 = QuaternionConv(channel_num, channel_num, 3, 1, padding=dilation, dilatation=dilation, groups=group, bias=False)
        self.norm2 = QuaternionInstanceNorm2d(channel_num)

    def forward(self, x):
        y = F.relu(self.norm1(self.conv1(self.pre_conv1(x))))
        y = self.norm2(self.conv2(self.pre_conv2(y)))
        return F.relu(x+y)

In [37]:
class ResidualBlock(nn.Module):
    def __init__(self, channel_num, dilation=1, group=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = QuaternionConv(channel_num, channel_num, 3, 1, padding=dilation, dilatation=dilation, groups=group, bias=False)
        self.norm1 = QuaternionInstanceNorm2d(channel_num)
        self.conv2 = QuaternionConv(channel_num, channel_num, 3, 1, padding=dilation, dilatation=dilation, groups=group, bias=False)
        self.norm2 = QuaternionInstanceNorm2d(channel_num)

    def forward(self, x):
        y = F.relu(self.norm1(self.conv1(x)))
        y = self.norm2(self.conv2(y))
        return F.relu(x+y)

In [38]:
class GCANet(nn.Module):
    def __init__(self, in_c=4, out_c=3, only_residual=True):
        super(GCANet, self).__init__()
        self.conv1 = QuaternionConv(in_c, 64, 3, 1, 1, 1, bias=False)
        self.norm1 = QuaternionInstanceNorm2d(64)
        self.conv2 = QuaternionConv(64, 64, 3, 1, 1, 1, bias=False)
        self.norm2 = QuaternionInstanceNorm2d(64)
        self.conv3 = QuaternionConv(64, 64, 3, 2, 1,1, bias=False)
        self.norm3 = QuaternionInstanceNorm2d(64)

        self.res1 = SmoothDilatedResidualBlock(64, dilation=2)
        self.res2 = SmoothDilatedResidualBlock(64, dilation=2)
        self.res3 = SmoothDilatedResidualBlock(64, dilation=2)
        self.res4 = SmoothDilatedResidualBlock(64, dilation=4)
        self.res5 = SmoothDilatedResidualBlock(64, dilation=4)
        self.res6 = SmoothDilatedResidualBlock(64, dilation=4)
        self.res7 = ResidualBlock(64, dilation=1)

        self.gate = QuaternionConv(64 * 3, 64, 1, 1, 1, bias=True)

        self.deconv3 = QuaternionTransposeConv(64, 64, 4, 2, 1, 1)
        self.norm4 = QuaternionInstanceNorm2d(64)
        self.deconv2 = QuaternionConv(2 * 64, 64, 3, 1, 1, 1)
        self.norm5 = QuaternionInstanceNorm2d(64)
        self.deconv1 = QuaternionConv(64, out_c, 1, 1, bias=True)
        self.only_residual = only_residual

    def forward(self, x):
        y = F.relu(self.norm1(self.conv1(x)))
        y0 = F.relu(self.norm2(self.conv2(y)))
        y1 = F.relu(self.norm3(self.conv3(y0)))

        y = self.res1(y1)
        y = self.res2(y)
        y = self.res3(y)
        y2 = self.res4(y)
        y = self.res5(y2)
        y = self.res6(y)
        y3 = self.res7(y)


        r = torch.cat((get_r(y1), get_r(y2), get_r(y3)), dim=1)
        i = torch.cat((get_i(y1), get_i(y2), get_i(y3)), dim=1)
        j = torch.cat((get_j(y1), get_j(y2), get_j(y3)), dim=1)
        k = torch.cat((get_k(y1), get_k(y2), get_k(y3)), dim=1)

        gated_y = self.gate(torch.cat((r, i, j, k), dim=1))

        y = F.relu(self.norm4(self.deconv3(gated_y)))   
        r = torch.cat((get_r(y0), get_r(y)), dim=1)
        i = torch.cat((get_i(y0), get_i(y)), dim=1)
        j = torch.cat((get_j(y0), get_j(y)), dim=1)
        k = torch.cat((get_k(y0), get_k(y)), dim=1)
        y = torch.cat((r, i, j, k), dim=1)     
        y = F.relu(self.norm5(self.deconv2(y)))
        if self.only_residual:
            y = self.deconv1(y)
        else:
            y = F.relu(self.deconv1(y))

        return y

# Perceptual loss

In [39]:
from torchvision.models import vgg16

In [40]:
class LossNetwork(torch.nn.Module):
    def __init__(self, vgg_model):
        super(LossNetwork, self).__init__()
        self.vgg_layers = vgg_model
        self.layer_name_mapping = {
            '3': "relu1_2",
            '8': "relu2_2",
            '15': "relu3_3"
        }

    def output_features(self, x):
        output = {}
        for name, module in self.vgg_layers._modules.items():
            x = module(x)
            if name in self.layer_name_mapping:
                output[self.layer_name_mapping[name]] = x
        return list(output.values())

    def forward(self, dehaze, gt):
        loss = []
        dehaze_features = self.output_features(dehaze)
        gt_features = self.output_features(gt)
        for dehaze_feature, gt_feature in zip(dehaze_features, gt_features):
            loss.append(F.mse_loss(dehaze_feature, gt_feature))

        return sum(loss)/len(loss)

In [41]:
# # --- Define the perceptual loss network --- #
# vgg_model = vgg16(pretrained=True).features[:16]
# vgg_model = vgg_model.cuda()
# for param in vgg_model.parameters():
#     param.requires_grad = False
# loss_network = LossNetwork(vgg_model)
# loss_network.eval()

# Train

In [43]:
def quat_l2(output, target):
    check_input(output)
    check_input(target)
    r = get_r(output) - get_r(target)
    i = get_i(output) - get_i(target)
    j = get_j(output) - get_j(target)
    k = get_k(output) - get_k(target)
    modulus = torch.sqrt(r*r + i*i + j*j + k*k)
    loss = torch.mean(modulus)
    return loss

In [45]:
opt = {'network': 'GCANet',
          'name': 'default_exp',
          'gpu_ids': '0',
          'epochs': 15, #100,
          'lr': 0.001,
          'lr_step': 6,
          'lr_gamma': 0.1,
          'weight_decay': 0.0005,
          'checkpoints_dir': 'checkpoint',
          'logDir': 'tblogdir',
          'resume_dir': '',
          'resume_epoch': 0,
          'save_epoch': 5,
          'save_latest_freq': 5000,
          'test_epoch': 5,
          'test_max_size': 1080,
          'size_unit': 8,
          'print_iter': 100,
          'input_folder': 'OTS/hazy',
          'gt_folder': 'OTS/gt',
          'test_input_folder': 'SOTS/outdoor/hazy',
          'test_gt_folder': 'SOTS/outdoor/gt',
          'num_workers': 4,
          'batch_size': 3,
          'only_residual': False,
          'loss_func': 'l2',
          'inc': 4,
          'outc': 4,
          'force_rgb': 'store_true',
          'no_edge':'store_true'}

opt['input_folder'] = os.path.expanduser(opt['input_folder'])
opt['gt_folder'] = os.path.expanduser(opt['gt_folder'])
opt['test_input_folder'] = os.path.expanduser(opt['test_input_folder'])
opt['test_gt_folder'] = os.path.expanduser(opt['test_gt_folder'])


if not os.path.exists(os.path.join(opt['checkpoints_dir'], opt['name'])):
    os.makedirs(os.path.join(opt['checkpoints_dir'], opt['name']))
opt['resume_dir'] = opt['resume_dir'] if opt['resume_dir'] != '' else os.path.join(opt['checkpoints_dir'], opt['name'])

# visualizer = TFVisualizer(opt)
# ### Log out
# with open(os.path.realpath(__file__), 'r') as fid:
#     visualizer.print_logs(fid.read())

# ## print argument
# for key, val in vars(opt).items():
#     visualizer.print_logs('%s: %s' % (key, val))

opt['gpu_ids'] = [int(x) for x in opt['gpu_ids'].split(',')]
assert all(0 <= x <= torch.cuda.device_count() for x in opt['gpu_ids']), 'gpu id should ' \
                                                      'be 0~{0}'.format(torch.cuda.device_count())
torch.cuda.set_device(opt['gpu_ids'][0])


train_dataset = ImagePairPrefixFolder(opt['input_folder'], opt['gt_folder'], size_unit=opt['size_unit'], force_rgb=opt['force_rgb'])
train_dataloader = DataLoader(train_dataset, batch_size=opt['batch_size'], shuffle=True,
                              collate_fn=var_custom_collate, pin_memory=True,
                              num_workers=opt['num_workers'])

opt['do_test'] = opt['test_gt_folder'] != ''
if opt['do_test']:
    test_dataset = ImagePairPrefixFolder(opt['test_input_folder'], 
                                         opt['test_gt_folder'],
                                         max_img_size=opt['test_max_size'], 
                                         size_unit=opt['size_unit'], 
                                         force_rgb=opt['force_rgb'])
    test_dataloader = DataLoader(test_dataset, 
                                 batch_size=1, 
                                 shuffle=False,
                                 collate_fn=var_custom_collate, 
                                 pin_memory=True,
                                 num_workers=1)

total_inc = opt['inc'] if opt['no_edge'] else opt['inc'] + 1
if opt['network'] == 'GCANet':
    net = GCANet(in_c=4, out_c=4, only_residual=opt['only_residual'])
else:
    print('network structure %s not supported' % opt['network'])
    raise ValueError

loss_crit = quat_l2
loss_crit2 = torch.nn.MSELoss()
loss_crit3 = torch.nn.L1Loss()
pnsr_crit = torch.nn.MSELoss()

if len(opt['gpu_ids']) > 0:
    net.cuda()
    if len(opt['gpu_ids']) > 1:
        net = torch.nn.DataParallel(net)
    pnsr_crit = pnsr_crit.cuda()

optimizer = optim.Adam(net.parameters(), lr=opt['lr'])
step_optim_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=opt['lr_step'], gamma=opt['lr_gamma'])
loss_avg = MovingAvg(pool_size=50)

start_epoch = 0
total_iter = 0

if os.path.exists(os.path.join(opt['checkpoints_dir'], opt['name'], 'latest.pth')):
    print('resuming from latest.pth')
    latest_info = torch.load(os.path.join(opt['checkpoints_dir'], opt['name'], 'latest.pth'))
    start_epoch = latest_info['epoch']
    total_iter = latest_info['total_iter']
    if isinstance(net, torch.nn.DataParallel):
        net.module.load_state_dict(latest_info['net_state'])
    else:
        net.load_state_dict(latest_info['net_state'])
    optimizer.load_state_dict(latest_info['optim_state'])

if opt['resume_epoch'] > 0:
    start_epoch = opt['resume_epoch']
    total_iter = opt['resume_epoch'] * len(train_dataloader)
    resume_path = os.path.join(opt['resume_epoch'], 'net_epoch_%d.pth') % opt['resume_epoch']
    print('resume from : %s' % resume_path)
    assert os.path.exists(resume_path), 'cannot find the resume model: %s ' % resume_path
    if isinstance(net, torch.nn.DataParallel):
        net.module.load_state_dict(torch.load(resume_path))
    else:
        net.load_state_dict(torch.load(resume_path))

for epoch in range(start_epoch, opt['epochs']):
    #visualizer.print_logs("Start to train epoch %d" % epoch)
    print("Start to train epoch %d" % epoch)
    net.train()
    for iter, data in enumerate(train_dataloader):
        total_iter += 1
        optimizer.zero_grad()
        step_optim_scheduler.step(epoch)

        batch_input_img, batch_input_edge,  batch_gt = data
        if len(opt['gpu_ids']) > 0:
            batch_input_img, batch_input_edge, batch_gt = batch_input_img.cuda(), batch_input_edge.cuda(), batch_gt.cuda()

        if opt['no_edge']:
            batch_input = batch_input_img
        else:
            batch_input = torch.cat((batch_input_img, batch_input_edge), dim=1)
   
        batch_input_v = Variable(batch_input)
        if opt['only_residual']:
            batch_gt_v = Variable(batch_gt - (batch_input_img+128))
        else:
            batch_gt_v = Variable(batch_gt)

        pred = net(batch_input_v)

        rec_loss1 = loss_crit(pred, batch_gt_v)
        rec_loss2 = loss_crit2(pred, batch_gt_v)
        rec_loss3 = loss_crit3(pred, batch_gt_v)
        #perceptual_loss = loss_network(pred[:, 1:, :, :], batch_gt_v[:, 1:, :, :])
        loss = (rec_loss1) * 0.33 + (rec_loss2)*0.33 + (rec_loss3)*0.33 #+ 0.04 *perceptual_loss


        loss.backward()
        optimizer.step()
        avg_loss = loss_avg.set_curr_val(loss.data)


        if iter % opt['print_iter'] == 0:
            # visualizer.plot_current_losses(total_iter, { 'loss': loss})
            # visualizer.print_logs('%s Step[%d/%d], lr: %f, mv_avg_loss: %f, loss: %f' %
            #                         (str(datetime.datetime.now()).split(' ')[1], iter, len(train_dataloader),
            #                          step_optim_scheduler.get_lr()[0], avg_loss, loss))
            pass

        if total_iter % opt['save_latest_freq'] == 0:
            latest_info = {'total_iter': total_iter,
                           'epoch': epoch,
                           'optim_state': optimizer.state_dict()}
            if len(opt['gpu_ids']) > 1:
                latest_info['net_state'] = net.module.state_dict()
            else:
                latest_info['net_state'] = net.state_dict()
            print('save lastest model.')
            torch.save(latest_info, os.path.join(opt['checkpoints_dir'], opt['name'], 'latest.pth'))

    if (epoch+1) % opt['save_epoch'] == 0 :
        #visualizer.print_logs('saving model for epoch %d' % epoch)
        if len(opt['gpu_ids']) > 1:
            torch.save(net.module.state_dict(), os.path.join(opt['checkpoints_dir'], opt['name'], 'net_epoch_%d.pth' % (epoch+1)))
        else:
            torch.save(net.state_dict(), os.path.join(opt['checkpoints_dir'], opt['name'], 'net_epoch_%d.pth' % (epoch + 1)))


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Start to train epoch 0


/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:154: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://git

RuntimeError: ignored

In [ ]:
NAME = 'old_new_model2'

In [ ]:
def restore_image(fname):
  img = Image.open(fname).convert('RGB')
  im_w, im_h = img.size
  if im_w % 4 != 0 or im_h % 4 != 0:
      img = img.resize((int(im_w // 4 * 4), int(im_h // 4 * 4))) 
  #img = np.array(img).astype('float')

  img = np.array(img)
  h, w, c = img.shape
  #gray = cv2.imread(fname.replace('hazy', 'trans').replace('.jpg','.png'), 0)
  #gray = cv2.resize(gray, (w, h))

  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  img = np.dstack([gray[:, :, np.newaxis], img])
  img = img.astype('float')

  img_data = torch.from_numpy(img.transpose((2, 0, 1))).float()
  c, w, h = img_data.size()
  #edge_data = torch.zeros((1, w, h))
  #in_data = torch.cat((img_data, edge_data), dim=0)
  #in_data = torch.cat((img_data, edge_data), dim=1).unsqueeze(0) - 128 
  in_data = img_data.reshape(1, 4, w, h) - 128
  in_data = in_data.cuda()
  with torch.no_grad():
      pred = net(Variable(in_data))

  out_img_data = (pred.data[0].cpu().float()).round().clamp(0, 255)
  out_img = out_img_data.numpy().astype(np.uint8).transpose(1, 2, 0)
  return out_img[:, :, 1:]

In [ ]:
import cv2
import numpy as np
from glob import glob
from skimage.metrics import peak_signal_noise_ratio
from skimage.metrics import structural_similarity

def image_colorfulness(image):
	# split the image into its respective RGB components
	(B, G, R) = cv2.split(image.astype("float"))
	# compute rg = R - G
	rg = np.absolute(R - G)
	# compute yb = 0.5 * (R + G) - B
	yb = np.absolute(0.5 * (R + G) - B)
	# compute the mean and standard deviation of both `rg` and `yb`
	(rbMean, rbStd) = (np.mean(rg), np.std(rg))
	(ybMean, ybStd) = (np.mean(yb), np.std(yb))
	# combine the mean and standard deviations
	stdRoot = np.sqrt((rbStd ** 2) + (ybStd ** 2))
	meanRoot = np.sqrt((rbMean ** 2) + (ybMean ** 2))
	# derive the "colorfulness" metric and return it
	return stdRoot + (0.3 * meanRoot)


!cp -r /content/drive/MyDrive/images .
!mkdir /content/drive/MyDrive/processed/{NAME}

for imname in glob('images/*.png'):
  reconstructed = restore_image(imname)
  fname = os.path.basename(imname)
  cv2.imwrite(os.path.join('/content/drive/MyDrive/processed/{}'.format(NAME), fname), cv2.cvtColor(reconstructed, cv2.COLOR_RGB2BGR))

hazy_dir = 'SOTS/outdoor/hazy/'
gt_dir = 'SOTS/outdoor/gt/'
hazy_imgs = [os.path.basename(fn) for fn in glob(os.path.join(hazy_dir, '*.jpg'))]


psnrs = []
ssims = []
colorfulls = []
for name in hazy_imgs:
  fname = os.path.basename(name).split("_")[0] + '.png'
  image_name = os.path.join(hazy_dir, name)
  gt_name = os.path.join(gt_dir, fname)

  reconstructed = restore_image(image_name)
  height, width = reconstructed.shape[:2]
  gt = cv2.imread(gt_name)
  gt = cv2.resize(gt, (width, height))
  gt = cv2.cvtColor(gt, cv2.COLOR_BGR2RGB)

  bgr = cv2.cvtColor(reconstructed, cv2.COLOR_RGB2BGR)

  cv2.imwrite(os.path.join('/content/drive/MyDrive/processed/{}'.format(NAME), fname), bgr)

  colorfulls.append(image_colorfulness(bgr))

  psnrs.append(peak_signal_noise_ratio(gt, reconstructed))

  img1 = cv2.cvtColor(reconstructed, cv2.COLOR_RGB2GRAY)
  img2 = cv2.cvtColor(gt, cv2.COLOR_RGB2GRAY)

  ssims.append(structural_similarity(img2, img1))



print('ssim:', sum(ssims)/len(ssims))
print('psnr:', sum(psnrs)/len(psnrs))
print('colorfullness:', sum(colorfulls)/len(colorfulls))

!mkdir /content/drive/MyDrive/networks/{NAME}
!cp -r checkpoint /content/drive/MyDrive/networks/{NAME}

In [ ]:
# for imname in glob('ds/real/hazy/*.png'):
#   print('imname')
#   reconstructed = restore_image(imname)
#   fname = os.path.basename(imname)
#   cv2.imwrite(os.path.join('/content/drive/MyDrive/processed/{}'.format(NAME), fname), cv2.cvtColor(reconstructed, cv2.COLOR_RGB2BGR))
